In [ ]:
import pandas as pd
import numpy as np
import pickle
import gzip
import os
import kaleido    # https://github.com/plotly/Kaleido
import plotly     # https://plotly.com/python/getting-started/
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "none"
pd.set_option('display.precision', 3)

## Load data

In [ ]:
CHM13_counts = pd.read_pickle('repeat_distributions/CHM13_counts.pickle')
random_counts = pd.read_pickle('repeat_distributions/random_counts.pickle')
subonly_counts = pd.read_pickle('repeat_distributions/subonly_counts.pickle')

denovo_exp_rate = pd.read_pickle('denovo/denovo_exp_rate.pickle')
denovo_con_rate = pd.read_pickle('denovo/denovo_con_rate.pickle')
denovo_nonexp_rate = pd.read_pickle('denovo/denovo_nonexp_rate.pickle')
denovo_substitution_context_rate = pd.read_pickle('denovo/denovo_mut_freq_triplets.pickle')
decode_exp_rate = pd.read_pickle('decode/decode_expansion_rates.pickle')
decode_con_rate = pd.read_pickle('decode/decode_contraction_rates.pickle')

repeats_1_4 = ['A', 'C', 'AC', 'AT', 'AG', 'CG', 'AAT', 'AAG', 'AAC', 'ATC', 'ACT', 'AGG', 'AGC', 'ACG', 'ACC', 'CCG', 'AAAT', 'AAAG', 'AAAC', 'AATG', 'AATC', 'AAGT', 'AAGG', 'AAGC', 'AACT', 'AACG', 'AACC', 'AGAT', 'ACAT', 'ATCC', 'ACAG', 'ACTC', 'ACTG', 'ACCT', 'AGGG', 'AGGC', 'AGCC', 'ACGG', 'ACGC', 'ACCG', 'ACCC', 'AGCG', 'CCCG', 'AATT', 'ATGC', 'ATCG', 'AGCT', 'ACGT', 'CCGG']

bootstrap_counts = pd.read_pickle('repeat_distributions/bootstrap_counts_1000.pickle').sort_index()
bootstrap_counts_max = dict(); bootstrap_counts_min = dict(); bootstrap_counts_mean = dict()
for motif in repeats_1_4:
    bootstrap_counts_mean[motif] = bootstrap_counts.xs(motif, level=1, drop_level=True, axis=1).apply(lambda x: x.sort_values().head(975).tail(950).mean(), axis=1).sort_index().replace(0, np.nan).dropna()
    bootstrap_counts_max[motif] = bootstrap_counts.xs(motif, level=1, drop_level=True, axis=1).apply(lambda x: x.sort_values().head(975).tail(950).max(), axis=1).reindex(bootstrap_counts_mean[motif].index)
    bootstrap_counts_min[motif] = bootstrap_counts.xs(motif, level=1, drop_level=True, axis=1).apply(lambda x: x.sort_values().head(975).tail(950).min(), axis=1).reindex(bootstrap_counts_mean[motif].index)

## Plot functions

In [ ]:
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]])
def make_colorscale(list_of_traces, opacity = 0.15):
    current_colors = pd.Series(['rgb('+str(current/len(list_of_traces)*255) + ', 180, '+str((len(list_of_traces)-current)/len(list_of_traces)*255)+')' for current in range(len(list_of_traces))], index = list_of_traces)
    return pd.Series(current_colors, index = list_of_traces), pd.Series(['rgba' + color[3:-1] + ', '+str(opacity)+')' for color in current_colors], index = list_of_traces)
def loop_list(input_list, count):
    return input_list * (count // len(input_list)) + input_list[:(count % len(input_list))]

In [ ]:
# add pseudocount to max_L bins, normalize, restrict to maxL bins
def curve_metric_to_startend(dist_1, dist_2 = None, dist2_grid = False, to_start = 'start', min_L = 0, max_L = 200, pseudocount = 1, normalize = False, keepsign = False, check = False):
    dist_1 = dist_1.dropna(how = 'all', axis=1).reindex(range(max_L)).fillna(0) + pseudocount
    if normalize == True:
        dist_1 = dist_1 / dist_1.sum()
    if to_start == 'start':
        dist_2 = dist_1[0]
    if to_start == 'end':
        dist_2 = dist_1[dist_1.columns.max()]
    if dist2_grid == True:
        dist_2 = dist_2.dropna(how = 'all', axis=1).reindex(range(max_L)).fillna(0) + pseudocount
        if normalize == True:
            dist_2 = dist_2 / dist_2.sum(axis=0)
    if to_start == False:
        if dist_2.index[0] == 1:
            dist_2.index = dist_2.index - 1
        dist_2 = dist_2.reindex(list(range(max_L))).fillna(0) + pseudocount
        if normalize == True:
            dist_2 = dist_2 / dist_2.sum()
    dist_1 = dist_1.reindex(list(range(min_L, max_L)))
    dist_2 = dist_2.reindex(list(range(min_L, max_L)))
    if check == True:
        print(dist_1)
        print(dist_2)
    if keepsign == False:
        if dist2_grid == True:
            return ((np.log(dist_1).sub(np.log(dist_2)))**2).sum(axis=0)
        else:
            return ((np.log(dist_1).sub(np.log(dist_2), axis=0))**2).sum()
    if keepsign == True:
        return (np.sign(dist_1.sub(dist_2, axis=0)) * ((np.log(dist_1).sub(np.log(dist_2), axis=0))**2)).sum()
    

In [ ]:
# evolve function
def mut_evolve_dist_AB(A_count_input, B_count_input, starting_conditions, boot = None, input_nuc = 'A', mut = True, mutonly = False, speedup_multiplier = 1, output_components = False, stochastics = None, reflective = True):
    exp_rate_A_AA, con_rate_A_AA, nonexp_rate_A_AB, B_indel_rates = starting_conditions
    A_count_output = A_count_input.copy(); B_count_output = B_count_input.copy()
    A_bins = len(A_count_input)
    B_bins = len(B_count_input)
    A_length_array = np.array(range(1,A_bins+3))
    A_length_array_bases = np.array(range(1,A_bins+3)) * len(input_nuc) ### including motif length
    B_length_array = np.array(range(1,B_bins+3))
    B_length_array_bases = np.array(range(1,B_bins+3)) * len(input_nuc) ### including motif length
    A_count_input = np.insert(A_count_input, A_bins, [0,0])
    B_count_input = np.insert(B_count_input, B_bins, [0,0])
    #A_count_input = A_count_input.astype('int64')
    #B_count_input = B_count_input.astype('int64')

    if boot is None:
        denovo_sub = denovo_substitution_context_rate.loc[input_nuc]
    else:
        denovo_sub = denovo_substitution_context_rate_poisson.loc[boot].loc[input_nuc]

    # distribution info
    total_B_bases = (B_count_input[:B_bins] * B_length_array_bases[:B_bins]).sum()
    B_L1_base_portion = ((B_count_input[0] * len(input_nuc)) / (B_count_input[:B_bins]* B_length_array_bases[:B_bins]).sum()) ### including motif length
    B_nonflank_base_portion = (B_count_input[2:B_bins+2] * B_length_array_bases[:B_bins]).sum() / total_B_bases  ### include portion of triplets 1nt away???
    B_flank_base_portion = (B_count_input[1:B_bins] * 2 * len(input_nuc)).sum() / total_B_bases ### including motif length
    
    total_A_bases = (A_count_input[:A_bins] * A_length_array_bases[:A_bins]).sum()
    A_nonflank_base_portion = (A_count_input[2:A_bins+2] * A_length_array_bases[:A_bins]).sum() / total_A_bases
    A_flank_base_portion = (A_count_input[1:A_bins] * 2 * len(input_nuc)).sum() / total_A_bases ### including motif length
    
    total_A_change_in = np.array([0.0]*A_bins); total_B_change_in = np.array([0.0]*B_bins)
    total_A_change_out = np.array([0.0]*A_bins); total_B_change_out = np.array([0.0]*B_bins)

    if mut == True:
        # A>B which adds to the A count locally. add these to A
        A_mut_in_local_A_B = 2 * len(input_nuc) * denovo_sub['Acontraction'] * A_count_input[1:]
        A_mut_out_local_A_B = -2 * len(input_nuc) * denovo_sub['Acontraction'] * A_count_input
        A_mut_out_local_A_B[0] = -1 * len(input_nuc) * A_count_input[0] * denovo_sub['A10']
        #A_mut_in_local_A_B = A_mut_out_local_A_B[1:]

        # total number of A>B fission events
        A_mut_out_fission = np.insert((-denovo_sub['Afission'] * A_count_input[2:] * A_length_array_bases[:A_bins]), 0, [0, 0]) # used to subtract from A_count, starting from L=3 (with 0 for L=1,2)
        # each fission creates 2 As. add these to A
        A_mut_in_fission =  np.array([np.sum(((2/A_length_array[:A_bins]) * -A_mut_out_fission[2:A_bins+2])[L-1:]) for L in A_length_array[:A_bins]]) ### use length_array_bases???
 
        # B>A which adds to the A count locally (which must come from B_L>1)
        # A from B>A leaving the -1 bin
        A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
        A_mut_out_local_B_A = -denovo_sub['Aexpansion'] * B_flank_base_portion * total_B_bases * A_len_freq
        # B>A creating A_L=1 from B_L>2
        B_A_into_L1 = total_B_bases * B_nonflank_base_portion * denovo_sub['A01']
        A_mut_in_local_B_A = np.insert(-A_mut_out_local_B_A, 0, B_A_into_L1)
        
        # fusion process for A
        A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
        A_fusion_freq_in = np.bincount((np.add.outer(A_length_array[:A_bins], A_length_array[:A_bins])+1).ravel(), weights = np.outer(A_len_freq, A_len_freq).ravel())[1:]
        A_mut_in_fusion_A_B = A_fusion_freq_in * denovo_sub['Afusion'] * B_L1_base_portion * total_B_bases
        A_mut_out_fusion_A_B = (-2) *A_len_freq * denovo_sub['Afusion'] * B_L1_base_portion * total_B_bases
        
        
        # total B>A
        # B>A which adds to the B count locally. add these to B
        B_mut_in_local_B_A = 2 * len(input_nuc) * denovo_sub['Aexpansion'] * B_count_input[1:]
        B_mut_out_local_B_A = -2 * len(input_nuc) * denovo_sub['Aexpansion'] * B_count_input
        B_mut_out_local_B_A[0] = -1 * B_L1_base_portion * total_B_bases * denovo_sub['Afusion']

        # total number of B>A fission events
        B_mut_out_fission = np.insert((-denovo_sub['A01'] * B_count_input[2:] * B_length_array_bases[:B_bins]), 0, [0, 0]) # used to subtract from B_count, starting from L=3 (with 0 for L=1,2)
        # each fission creates 2 Bs. add these to B
        B_mut_in_fission =  np.array([np.sum(((2/B_length_array[:B_bins]) * -B_mut_out_fission[2:B_bins+2])[L-1:]) for L in B_length_array[:B_bins]]) ### use length_array_bases???

        # A>B which adds to the B count locally (which must come from A_L>1)
        # B from A>B leaving the -1 bin
        B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
        B_mut_out_local_A_B = -denovo_sub['Acontraction'] * A_flank_base_portion * total_A_bases * B_len_freq
        # A>B creating B_L=1 from A_L>2
        A_B_into_L1 = total_A_bases * A_nonflank_base_portion * denovo_sub['Afission']
#        A_B_into_L1 = -A_mut_out_fission.sum()
        B_mut_in_local_A_B = np.insert(-B_mut_out_local_A_B, 0, A_B_into_L1)
        
        # fusion process for B
        B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
        B_fusion_freq_in = np.bincount((np.add.outer(B_length_array[:B_bins], B_length_array[:B_bins])+1).ravel(), weights = np.outer(B_len_freq, B_len_freq).ravel())[1:]
        B_mut_in_fusion_B_A = B_fusion_freq_in * denovo_sub['A10'] * A_count_input[0] * len(input_nuc)
        B_mut_out_fusion_B_A = (-2) * B_len_freq * denovo_sub['A10'] * A_count_input[0] * len(input_nuc)

        # update counts for next round (with absorbing boundary)
        total_A_change_in += A_mut_in_local_A_B[:A_bins] + A_mut_in_local_B_A[:A_bins] + A_mut_in_fission[:A_bins] + A_mut_in_fusion_A_B[:A_bins]
        total_B_change_in += B_mut_in_local_B_A[:B_bins] + B_mut_in_local_A_B[:B_bins] + B_mut_in_fission[:B_bins] + B_mut_in_fusion_B_A[:B_bins]
        total_A_change_out += A_mut_out_local_A_B[:A_bins] + A_mut_out_local_B_A[:A_bins] + A_mut_out_fission[:A_bins] + A_mut_out_fusion_A_B[:A_bins]
        total_B_change_out += B_mut_out_local_B_A[:B_bins] + B_mut_out_local_A_B[:B_bins] + B_mut_out_fission[:B_bins] + B_mut_out_fusion_B_A[:B_bins]

        # apply reflecting boundary
        if reflective == True:
            total_A_change_in[A_bins-1] += A_mut_in_local_A_B[A_bins:].sum() + A_mut_in_local_B_A[A_bins:].sum() + A_mut_in_fission[A_bins:].sum() + A_mut_in_fusion_A_B[A_bins:].sum()
            total_B_change_in[B_bins-1] += B_mut_in_local_B_A[B_bins:].sum() + B_mut_in_local_A_B[B_bins:].sum() + B_mut_in_fission[B_bins:].sum() + B_mut_in_fusion_B_A[B_bins:].sum()
            total_A_change_out[A_bins-1] += A_mut_out_local_A_B[A_bins:].sum() + A_mut_out_local_B_A[A_bins:].sum() + A_mut_out_fission[A_bins:].sum() + A_mut_out_fusion_A_B[A_bins:].sum()
            total_B_change_out[B_bins-1] += B_mut_out_local_B_A[B_bins:].sum() + B_mut_out_local_A_B[B_bins:].sum() + B_mut_out_fission[B_bins:].sum() + B_mut_out_fusion_B_A[B_bins:].sum()
           
            
    if mutonly == False:
        # A expansions in and out
        A_exp_out = A_count_input[:A_bins] * -exp_rate_A_AA[:A_bins]
        A_exp_in = np.insert(-A_exp_out, 0, B_indel_rates[0]*total_B_bases)

        # A contractions in and out
        A_con_out = A_count_input[:A_bins+1] * -con_rate_A_AA[:A_bins+2]
        A_con_in = -A_con_out[1:]

        # A fusions from B1>B0 deletions
        if (mut != True):
            A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
            A_fusion_freq_in = np.bincount((np.add.outer(A_length_array[:A_bins], A_length_array[:A_bins])+1).ravel(), weights = np.outer(A_len_freq, A_len_freq).ravel())[1:]
        A_mut_in_fusion_Bdel = A_fusion_freq_in[1:A_bins+1] * B_indel_rates[1] * B_L1_base_portion * total_B_bases
        A_mut_out_fusion_Bdel = (-2) *A_len_freq * B_indel_rates[1] * B_L1_base_portion * total_B_bases

        # A fission events from insertions
        A_nonexp_fissions_out = -A_count_input * nonexp_rate_A_AB # used to calculate fission_in, starting with L=2 going to 2x L=1
        # each fission creates 2 As. add these to A
        A_nonexp_in_fission =  np.array([np.sum(((2/A_length_array[:A_bins]) * -A_nonexp_fissions_out[1:A_bins+1])[L-1:]) for L in A_length_array[:A_bins]])


        # B expansions in and out
        B_exp_out = B_count_input[:B_bins] * -B_indel_rates[2] * B_length_array[:B_bins] # B>BB rates are flat, per base
        B_exp_in = np.insert(-B_exp_out, 0, A_nonexp_fissions_out.sum())

        # B contractions in and out
        B_con_out = B_count_input[:B_bins+1] * -B_indel_rates[1] * B_length_array[:B_bins+1] # B>_ rates are flat, per base
        B_con_in = -B_con_out[1:]

        # B fusions from A1>A0 deletions
        if (mut != True):
            B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
            B_fusion_freq_in = np.bincount((np.add.outer(B_length_array[:B_bins], B_length_array[:B_bins])+1).ravel(), weights = np.outer(B_len_freq, B_len_freq).ravel())[1:]
        B_mut_in_fusion_Adel = B_fusion_freq_in[1:B_bins+1] * A_count_input[0] * con_rate_A_AA[0]
        B_mut_out_fusion_Adel = 2 *B_len_freq * A_count_input[0] * -con_rate_A_AA[0]
                    
        # B fission events from insertions
        B_nonexp_fissions_out = -B_count_input * B_indel_rates[0] * B_length_array # used to calculate fission_in, starting with L=2 going to 2x L=1
        # each fission creates 2 Bs. add these to B
        B_nonexp_in_fission =  np.array([np.sum(((2/B_length_array[:B_bins]) * -B_nonexp_fissions_out[1:B_bins+1])[L-1:]) for L in B_length_array[:B_bins]])
            
       # update counts for next round (with absorbing boundary)
        total_A_change_in += A_exp_in[:A_bins] + A_con_in[:A_bins] + A_mut_in_fusion_Bdel[:A_bins] + A_nonexp_in_fission[:A_bins]
        total_B_change_in += B_exp_in[:B_bins] + B_con_in[:B_bins] + B_mut_in_fusion_Adel[:B_bins] + B_nonexp_in_fission[:B_bins]
        total_A_change_out += A_exp_out[:A_bins] + A_con_out[:A_bins] + A_mut_out_fusion_Bdel[:A_bins] + A_nonexp_fissions_out[:A_bins]
        total_B_change_out += B_exp_out[:B_bins] + B_con_out[:B_bins] + B_mut_out_fusion_Adel[:B_bins] + B_nonexp_fissions_out[:B_bins]

        # apply reflecting boundary
        if reflective == True:
            total_A_change_in[A_bins-1] += A_exp_in[A_bins:].sum() + A_con_in[A_bins:].sum() + A_mut_in_fusion_Bdel[A_bins:].sum() + A_nonexp_in_fission[A_bins:].sum()
            total_B_change_in[B_bins-1] += B_exp_in[B_bins:].sum() + B_con_in[B_bins:].sum() + B_mut_in_fusion_Adel[B_bins:].sum() + B_nonexp_in_fission[B_bins:].sum()
            total_A_change_out[A_bins-1] += A_exp_out[A_bins:].sum() + A_con_out[A_bins:].sum() + A_mut_out_fusion_Bdel[A_bins:].sum() + A_nonexp_fissions_out[A_bins:].sum()
            total_B_change_out[B_bins-1] += B_exp_out[B_bins:].sum() + B_con_out[B_bins:].sum() + B_mut_out_fusion_Adel[B_bins:].sum() + B_nonexp_fissions_out[B_bins:].sum()

    
    # flag to stop the simulation if more repeats are removed from a bin than exist in that bin (excluding the last 10 noisy bins)
    flag = ((np.abs(total_A_change_out[:A_bins-10]) * speedup_multiplier > A_count_output[:A_bins-10]).sum()) > 0

    # apply speedup
    total_A_change_in *= speedup_multiplier; total_A_change_out *= speedup_multiplier
    total_B_change_in *= speedup_multiplier; total_B_change_out *= speedup_multiplier
    
    if stochastics is not None:
        # the sum of poisson random variables is poisson-distributed. not necessary to run n poisson samples
        total_A_change_in = np.random.poisson(total_A_change_in)
        total_A_change_out = -1 * np.random.poisson(np.abs(total_A_change_out))
        total_B_change_in = np.random.poisson(total_B_change_in)
        total_B_change_out = -1 * np.random.poisson(np.abs(total_B_change_out))   
    
    total_A_change = total_A_change_in + total_A_change_out
    total_B_change = total_B_change_in + total_B_change_out
    
    # update counts for next round
    A_count_output = A_count_output[:A_bins] + total_A_change[:A_bins]
    B_count_output = B_count_output[:B_bins] + total_B_change[:B_bins]

    # remove negative values
    A_count_output[A_count_output <0] = 0            
    B_count_output[B_count_output <0] = 0
    
    if output_components == True:
        if mutonly == False:
            return  A_mut_in_local_A_B[:A_bins], A_mut_out_local_A_B[:A_bins], A_mut_in_local_B_A[:A_bins], A_mut_out_local_B_A[:A_bins], A_mut_in_fission[:A_bins], A_mut_out_fission[:A_bins], A_mut_in_fusion_A_B[:A_bins], A_mut_out_fusion_A_B[:A_bins], A_exp_in[:A_bins], A_exp_out[:A_bins], A_con_in[:A_bins], A_con_out[:A_bins], A_mut_in_fusion_Bdel[:A_bins], A_mut_out_fusion_Bdel[:A_bins], A_nonexp_in_fission[:A_bins], A_nonexp_fissions_out[:A_bins]
        else:
            return  A_mut_in_local_A_B[:A_bins], A_mut_out_local_A_B[:A_bins], A_mut_in_local_B_A[:A_bins], A_mut_out_local_B_A[:A_bins], A_mut_in_fission[:A_bins], A_mut_out_fission[:A_bins], A_mut_in_fusion_A_B[:A_bins], A_mut_out_fusion_A_B[:A_bins]
    else:
        return A_count_output, B_count_output, flag

In [ ]:
def extend_power_law(power, start_rate, start_len, end_len=200):
    denom = (start_len**power) / start_rate
    return pd.Series([i**power for i in range(start_len+1, end_len+1)], index = list(range(start_len+1,end_len+1))) / denom

def multiply_then_powerlaw(exp_power, con_power, mult, A_bins = 200, boot = None, L_mult = 9, L_mult_nonexp = 9, motif = 'A', fill = False, nonexp_factor = False):
    if boot is None:
        bootname = ''
        denovo_exp_rate_current = denovo_exp_rate[motif]
        denovo_con_rate_current = denovo_con_rate[motif]
        denovo_nonexp_rate_current = denovo_nonexp_rate[motif]
    else:
        bootname = '_boot'+str(boot)
        denovo_exp_rate_current = denovo_exp_rate_poisson[motif][boot]
        denovo_con_rate_current = denovo_con_rate_poisson[motif][boot]
        denovo_nonexp_rate_current = denovo_nonexp_rate_poisson[motif][boot]
    if fill == True:
        fillname = '_fill'
        denovo_exp_rate_current = denovo_exp_rate_current.replace(0, np.nan).interpolate(method = 'from_derivatives')
        denovo_con_rate_current = denovo_con_rate_current.replace(0, np.nan).interpolate(method = 'from_derivatives')
        denovo_nonexp_rate_current = denovo_nonexp_rate_current.replace(0, np.nan).interpolate(method = 'from_derivatives')
    else:
        fillname = ''
    exp = pd.concat([denovo_exp_rate_current.reindex(range(L_mult)), pd.Series(denovo_exp_rate_current[L_mult-1] * mult, index = [L_mult]), extend_power_law(exp_power, denovo_exp_rate_current[L_mult-1]*mult, L_mult, A_bins+3)])
    con = pd.concat([denovo_con_rate_current.reindex(range(L_mult)), pd.Series(denovo_con_rate_current[L_mult-1] * mult, index = [L_mult]), extend_power_law(con_power, denovo_con_rate_current[L_mult-1]*mult, L_mult, A_bins+3)])
    if nonexp_factor == False:
        nonexpname = ''
        nonexp = pd.concat([denovo_nonexp_rate_current.reindex(range(L_mult_nonexp)), pd.Series(denovo_nonexp_rate_current[L_mult_nonexp-1] * mult, index = [L_mult_nonexp]), extend_power_law(exp_power, denovo_nonexp_rate_current[L_mult_nonexp-1]*mult, L_mult_nonexp, A_bins+3)])
    else:
        nonexpname = '_nonexp_x' + str(nonexp_factor)
        nonexp = exp * nonexp_factor
    nonexp.loc[1] = 0
    name = 'mult_L'+str(L_mult)+'_x'+str(mult)+'_extend_pl_' + str(exp_power) + '_' + str(con_power) + nonexpname + fillname + bootname
    return name, exp, con, nonexp

In [ ]:
# setup function
def setup_evolve(exp_power=1, con_power=1, mult=1, boot = None, stochastics = None, L_mult = 9, L_mult_nonexp = 9, fill = False, nonexp_factor = False, A_bins = 200, B_bins = 200, input_nuc = 'A', mutonly = False, exp_zero = False, con_zero = False, nonexp_zero = False, different_input = False, random_start = False, ceiling = None):
# set up counts
    A_length_array = np.array(range(1,A_bins+1))
    B_length_array = np.array(range(1,A_bins+1))
    if different_input == False:
        if random_start == False:
            A_count_input = np.nan_to_num(CHM13_counts['A'][input_nuc].reindex(range(1,A_bins+1)).values)
            B_count_input = np.nan_to_num(CHM13_counts['B'][input_nuc].reindex(range(1,B_bins+1)).values)
        if random_start == True:
            A_count_input = np.nan_to_num(random_counts['A'][input_nuc].reindex(range(1,A_bins+1)).values)
            B_count_input = np.nan_to_num(random_counts['B'][input_nuc].reindex(range(1,B_bins+1)).values)
    else:
        A_count_input = np.nan_to_num(different_input[0].reindex(range(1,A_bins+1)).values)
        B_count_input = np.nan_to_num(different_input[1].reindex(range(1,B_bins+1)).values)
# set up rates    
    if mutonly == False:
        name, exp_rate, con_rate, nonexp_rate = multiply_then_powerlaw(exp_power = exp_power, con_power = con_power, A_bins = A_bins, mult=mult, boot = boot, L_mult = 9, L_mult_nonexp = 9, motif = input_nuc, fill = fill, nonexp_factor = nonexp_factor)
        B_indel_rate = np.array([exp_rate[0], con_rate[0], nonexp_rate[0]])        
        # change rates from per unit to per STR
        exp_rate = exp_rate.values[1:A_bins+1] * A_length_array
        con_rate = con_rate.values[1:A_bins+2] * np.array(range(1,A_bins+2))
        nonexp_rate = nonexp_rate.values[1:A_bins+3] * np.array(range(1,A_bins+3))
        if ceiling != None:
            exp_rate[exp_rate > ceiling] = ceiling
            con_rate[con_rate > ceiling] = ceiling
            nonexp_rate[nonexp_rate > ceiling] = ceiling
            name = name + '_ceiling_' + str(ceiling)
        if exp_zero == True:
            exp_rate *= 0
        if con_zero == True:
            con_rate *= 0
        if nonexp_zero == True:
            nonexp_rate *= 0
        if random_start == False:
            name = name + '_CHM13start'
        if random_start == True:
            name = name + '_randomstart'
        if stochastics is None:
            name = name
        else:
            name = name + '_stochastics_' + str(stochastics)
        return A_count_input, B_count_input, exp_rate, con_rate, nonexp_rate, B_indel_rate, name
    else:
        if boot is None:
            name = 'mutonly'
        else:
            name = 'mutonly_boot' + str(boot)
        if random_start == False:
            name = name + '_CHM13start'
        if random_start == True:
            name = name + '_randomstart'
        if stochastics is None:
            name = name
        else:
            name = name + '_stochastics_' + str(stochastics)
        return A_count_input, B_count_input, None, None, None, None, name

In [ ]:
def plot_relative_processes(mult, exp, con, different_input, L_mult=9, motif = 'A', show = False, norm = True, returndata = False, separate = False, A_bins = 200, B_bins = 200, cut_at_n1 = False):
    starting_conditions = setup_evolve(exp, con, mult, input_nuc = motif, different_input = different_input, A_bins = A_bins, B_bins = B_bins, L_mult=L_mult)
    A_components = mut_evolve_dist_AB(different_input[0][:A_bins], different_input[1][:B_bins], starting_conditions[2:6], output_components=True, input_nuc = motif, speedup_multiplier = 0)
    A_components = pd.DataFrame(A_components)
    A_components.columns = A_components.columns + 1
    A_components.index = ['mut_in_local_A_B', 'mut_out_local_A_B', 'mut_in_local_B_A', 'mut_out_local_B_A', 'mut_in_fission', 'mut_out_fission', 'mut_in_fusion_A_B', 'mut_out_fusion_A_B', 'exp_in', 'exp_out', 'con_in', 'con_out', 'mut_in_fusion_Bdel', 'mut_out_fusion_Bdel', 'nonexp_in_fission', 'nonexp_fissions_out']
    A_components_in = A_components.loc[A_components.index[::2]]
    A_components_out = A_components.loc[A_components.index[1::2]]
#    A_components_in.index = ['sub. -1', 'sub. +1', 'sub. fission', 'sub. fusion', 'exp. +1', 'con. -1', 'del. fusion', 'ins. fission']
#    A_components_out.index = ['sub. -1', 'sub. +1', 'sub. fission', 'sub. fusion', 'exp. +1', 'con. -1', 'del. fusion', 'ins. fission']
    A_components_in.index = ['substitution -1', 'substitution +1', 'substitution fission', 'substitution fusion', 'expansion +1', 'contraction -1', 'deletion fusion', 'non-motif insertion fission']
    A_components_out.index = ['substitution -1', 'substitution +1', 'substitution fission', 'substitution fusion', 'expansion +1', 'contraction -1', 'deletion fusion', 'non-motif insertion fission']
    A_components_sum = A_components_in + A_components_out
    if norm == True:
        abs_total = A_components_in.abs().sum() + A_components_out.abs().sum()
        A_components_in = A_components_in / abs_total
        A_components_out = A_components_out / abs_total
        A_components_sum = A_components_sum / A_components_sum.abs().sum()
    if returndata == 'sum':
        return A_components_sum
    if returndata == 'in':
        return A_components_in
    if returndata == 'out':
        return A_components_out
    else:
        last_int_bin = (different_input[0] >= 1).idxmin()
        fig_components = go.Figure()
        if separate == True:
            for component in A_components_sum.index:
                fig_components.add_trace(go.Bar(name = component, x = A_components_in.columns, y = A_components_in.loc[component], legendgroup = component))
            for component in A_components_sum.index:
                fig_components.add_trace(go.Bar(name = component, x = A_components_out.columns, y = A_components_out.loc[component], legendgroup = component, showlegend = False))
        else:
            for component in A_components_sum.index:
                fig_components.add_trace(go.Bar(name = component, x = A_components_sum.columns, y = A_components_sum.loc[component]))
        fig_components.add_trace(go.Scatter(x = [last_int_bin] *2, y = [A_components_sum.min().min() - 0.05, A_components_sum.max().max() + 0.1], name = 'count <1', mode = 'lines', line = dict(color = 'rgba(0,0,0,0.5)', dash = 'dash', width = 3)))
        fig_components.update_layout(barmode='relative', colorway = plotly.colors.qualitative.Plotly[:8], height = 250, margin={'t':20,'l':60,'b':40,'r':10})        
        if cut_at_n1 == True:
            if last_int_bin > 10:
                fig_components.update_xaxes(range = [0.5, last_int_bin + 0.5], title = 'repeat length (nt)')
        if cut_at_n1 == 'sup':
            if last_int_bin > 10:
                fig_components.update_xaxes(range = [0.5, last_int_bin + 0.5], title = 'repeat length (nt)')
        else:
            fig_components.update_xaxes(range = [0.5, A_bins + 0.5], title = 'repeat length (nt)')
        fig_components.update_yaxes(title = 'relative contribution', gridcolor = 'rgba(0,0,0,0.5)')
        if show == True:
            fig_components.show()
        return fig_components, last_int_bin

## Load files

In [ ]:
folder = 'simulations/grid_A_L9_subonlystart_prospeedup_200/'

#### Simple job list

In [ ]:
jobs = pd.DataFrame([(mult, exp/10, con/10) for mult in [1,2,4,8,16,32] for exp in range(51) for con in range(51)])
jobs.reset_index(inplace = True)
jobs.columns = ['ArrayTaskID', 'mult', 'exp', 'con']
jobs['ArrayTaskID'] += 1
jobs['mult'] = jobs['mult'].astype(float)
jobs.to_csv(folder + 'jobs.csv', index = False, sep = '\t')
jobs2 = jobs[10000:].copy()
jobs2['ArrayTaskID'] -=10000
jobs2.to_csv(folder + 'jobs2.csv', index = False, sep = '\t')
jobs2

In [ ]:
jobs3 = pd.DataFrame([(mult, exp/10, con/10) for mult in [1,2,4,8,16,32] for exp in range(26) for con in range(26)])
jobs3.reset_index(inplace = True)
jobs3.columns = ['ArrayTaskID', 'mult', 'exp', 'con']
jobs3['ArrayTaskID'] += 1
jobs3['mult'] = jobs3['mult'].astype(float)
jobs3.to_csv(folder + 'jobs_short.csv', index = False, sep = '\t')

#### Load simulation data

In [ ]:
grid_files = os.listdir(folder)
grid_files_A = [file for file in grid_files if 'Adist' in file]

parameter_info = pd.Series(grid_files_A).str.split('_', expand = True)[[2,3,4,6,7,10,11,12]]
parameter_info.index = grid_files_A
parameter_info[2] = parameter_info[2].str.split('bins', expand = True)[1].astype(int)
parameter_info[3] = parameter_info[3].str.split('sp1e', expand = True)[1].astype(int)
#parameter_info[4] = parameter_info[4].str.split('rounds1e', expand = True)[1].astype(int)
parameter_info[6] = parameter_info[6].str.split('L', expand = True)[1].astype(int)
parameter_info[7] = parameter_info[7].str.split('x', expand = True)[1].astype(float)
parameter_info[10] = parameter_info[10].astype(float)
parameter_info[11] = parameter_info[11].astype(float)
parameter_info[12] = parameter_info[12].str.split('.pickle', expand = True)[0]
parameter_info.columns = ['bins', 'speedup (1e)', 'rounds (1e)', 'L_mult', 'mult', 'exp', 'con', 'start']

In [ ]:
parameter_info['speedup (1e)'].value_counts()

#### Check for missing files
re-run from "Load files" after filling in any missing files

In [ ]:
jobs_incomplete = jobs.set_index(['mult', 'exp', 'con']).reindex(parameter_info.set_index(['mult', 'exp', 'con']).reindex(jobs.set_index(['mult', 'exp', 'con']).index).fillna('missing').replace('prospeedup', np.nan).dropna().index).reset_index()

In [ ]:
jobs_incomplete.reset_index(inplace = True)
jobs_incomplete.columns = ['ArrayTaskID', 'mult', 'exp', 'con', 'oldid']
jobs_incomplete = jobs_incomplete[['ArrayTaskID', 'mult', 'exp', 'con']]
jobs_incomplete['ArrayTaskID'] += 1
jobs_incomplete['mult'] = jobs_incomplete['mult'].astype(float)
jobs_incomplete.to_csv('simulations/grid_A_L9_subonlystart_prospeedup/jobs_incomplete.csv', index = False, sep = '\t')
jobs_incomplete

#### Load simulation data

In [ ]:
parameters_inference_cols = parameter_info.groupby(['speedup (1e)', 'rounds (1e)', 'mult', 'exp', 'con', 'start'])['con'].count().index
inference_grid = dict()
for speedup, rounds, mult, exp, con, start in parameters_inference_cols:
    file = 'Adist_A_bins200_sp1e0_prospeedup_mult_L9_x'+str(mult)+'_extend_pl_'+str(exp)+'_'+str(con)+'_'+start+'.pickle'
    inference_grid[(mult, exp, con)] = pd.read_pickle(folder + file)
    inference_grid[(mult, exp, con)].columns = inference_grid[(mult, exp, con)].columns * (10**speedup)
inference_grid = pd.concat(inference_grid, axis=1).sort_index(axis=1)

In [ ]:
inference_grid_final = inference_grid.T.groupby(level=[0,1,2]).tail(1).transpose()
inference_grid_final.columns = inference_grid_final.columns.droplevel(3)

In [ ]:
# export grid final timepoint for comparison to analytics
inference_grid_final.index +=1
inference_grid_final.to_csv(folder+'/distributions_grid_prospeedup_Lbound200.csv')
inference_grid_final.index -=1

## Normalized metric grid

In [ ]:
metric_inference_grid = curve_metric_to_startend(inference_grid, dist_2 = bootstrap_counts_mean['A'].copy(), to_start = False, max_L = 200, normalize=True).unstack().transpose()
metric_inference_grid_final = metric_inference_grid.apply(lambda x: x[x.notnull()].values[-1])

In [ ]:
metric_inference_grid_final.to_csv(folder+'/metric_grid_prospeedup_finaltimepoint.csv')

In [ ]:
last_bin = (inference_grid_final > 1).idxmin(axis=0).replace(0,200)

def find_max_rate(mult, exp, con, last_bin):
    return extend_power_law(exp, denovo_exp_rate['A'][8] *mult, 9, 203)[last_bin] + extend_power_law(con, denovo_con_rate['A'][8]  *mult, 9, 203)[last_bin]

max_rate = last_bin.reset_index()
max_rate.columns = ['mult', 'exp', 'con', 'last_bin']
max_rate['max_rate'] = max_rate.apply(lambda x: find_max_rate(x.mult, x.exp, x.con, x.last_bin), axis=1)
max_rate['max_rate_above01'] = max_rate['max_rate'] > 0.1

colorscale_clear_white=[[0.0, 'rgba(230,230,230,0)'], [1.0, 'rgba(230,230,230,1)']]

In [ ]:
fig = make_subplots(rows = 2, cols = 3, shared_xaxes = True, shared_yaxes = True, subplot_titles = ['<i>m</i>=1','<i>m</i>=2','<i>m</i>=4', '<i>m</i>=8','<i>m</i>=16','<i>m</i>=32'], vertical_spacing = 0.12, horizontal_spacing = 0.05, row_heights = [0.5, 0.5], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for mult in [1,2,4]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = metric_inference_grid_final[mult].index.get_level_values(0), y = metric_inference_grid_final[mult].index.get_level_values(1), z=np.log10(metric_inference_grid_final[mult]), coloraxis='coloraxis1'), row = 1, col = col_count)
    fig.add_trace(go.Heatmap(x = max_rate.loc[(max_rate['mult'] == mult)]['exp'], y = max_rate.loc[(max_rate['mult'] == mult)]['con'], z = max_rate.loc[(max_rate['mult'] == mult)]['max_rate_above01'].astype(int), showlegend = False, coloraxis='coloraxis2'), row = 1, col = col_count)
col_count = 0
for mult in [8,16,32]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = metric_inference_grid_final[mult].index.get_level_values(0), y = metric_inference_grid_final[mult].index.get_level_values(1), z=np.log10(metric_inference_grid_final[mult]), coloraxis='coloraxis1'), row = 2, col = col_count)
    fig.add_trace(go.Heatmap(x = max_rate.loc[(max_rate['mult'] == mult)]['exp'], y = max_rate.loc[(max_rate['mult'] == mult)]['con'], z = max_rate.loc[(max_rate['mult'] == mult)]['max_rate_above01'].astype(int), showlegend = False, coloraxis='coloraxis2'), row = 2, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(title = 'log<sub>10</sub>(metric)', len=1), cmin = np.log10(metric_inference_grid.loc[metric_inference_grid.index.max()].min()), cmax= 3.55))
fig.update_layout(coloraxis2=dict(colorscale=colorscale_clear_white, showscale = False, cmin = 0, cmax= 1))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 580, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,5], dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,5], dtick = 2, col = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,5], dtick = 2, col = 2, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,5], dtick = 2, col = 3, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.add_shape(type="rect", x0=2.369-0.745, y0=2.879-0.53, x1=2.369+0.745, y1=2.879+0.53, line=dict(color='rgba(0,255,255,0.85)'), row = 2, col = 1)
fig.add_shape(type="rect", x0=2.369-0.745, y0=2.879-0.53, x1=2.369+0.745, y1=2.879+0.53, line=dict(color='rgba(0,255,255,0.85)'), row = 2, col = 2)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/fig2b.svg')
fig.write_image('plots/fig2b.pdf')
fig.write_image('plots/fig2b.png', scale = 10)

In [ ]:
metric_inference_grid_norm_sorted = metric_inference_grid_final.sort_values()

In [ ]:
metric_inference_grid_norm_sorted.reset_index()[:10]

### Grid with 1e9 speedup

In [ ]:
folder_1e9 = 'simulations/grid_A_L9_subonlystart_long/'
grid_files = os.listdir(folder_1e9)
grid_files_A = [file for file in grid_files if 'Adist' in file]

parameter_info_1e9 = pd.Series(grid_files_A).str.split('_', expand = True)[[2,3,4,6,7,10,11,12]]
parameter_info_1e9.index = grid_files_A
parameter_info_1e9[2] = parameter_info_1e9[2].str.split('bins', expand = True)[1].astype(int)
parameter_info_1e9[3] = parameter_info_1e9[3].str.split('sp1e', expand = True)[1].astype(int)
parameter_info_1e9[4] = parameter_info_1e9[4].str.split('rounds1e', expand = True)[1].astype(int)
parameter_info_1e9[6] = parameter_info_1e9[6].str.split('L', expand = True)[1].astype(int)
parameter_info_1e9[7] = parameter_info_1e9[7].str.split('x', expand = True)[1].astype(float)
parameter_info_1e9[10] = parameter_info_1e9[10].astype(float)
parameter_info_1e9[11] = parameter_info_1e9[11].astype(float)
parameter_info_1e9[12] = parameter_info_1e9[12].str.split('.pickle', expand = True)[0]
parameter_info_1e9.columns = ['bins', 'speedup (1e)', 'rounds (1e)', 'L_mult', 'mult', 'exp', 'con', 'start']

# sort by speedup and remove (faster) duplicates
parameter_info_1e9 = parameter_info_1e9.sort_values(by = 'speedup (1e)').drop_duplicates(subset = ['mult', 'exp', 'con'], keep = 'first')

In [ ]:
parameter_info_1e9 = parameter_info_1e9.loc[(parameter_info_1e9['exp'] <= 3) & (parameter_info_1e9['con'] <= 3)]

In [ ]:
parameters_inference_cols = parameter_info_1e9.groupby(['speedup (1e)', 'rounds (1e)', 'mult', 'exp', 'con', 'start'])['con'].count().index
inference_grid_1e9 = dict()
for speedup, rounds, mult, exp, con, start in parameters_inference_cols:
    file = 'Adist_A_bins100_sp1e'+str(speedup)+'_rounds1e'+str(rounds)+'_mult_L9_x'+str(mult)+'_extend_pl_'+str(exp)+'_'+str(con)+'_'+start+'.pickle'
    inference_grid_1e9[(mult, exp, con)] = pd.read_pickle(folder_1e9 + file)
    inference_grid_1e9[(mult, exp, con)].columns = inference_grid_1e9[(mult, exp, con)].columns * (10**speedup)
inference_grid_1e9 = pd.concat(inference_grid_1e9, axis=1).sort_index(axis=1)

In [ ]:
inference_grid_1e9_final = inference_grid_1e9.T.groupby(level=[0,1,2]).tail(1).transpose()
inference_grid_1e9_final.columns = inference_grid_1e9_final.columns.droplevel(3)

In [ ]:
metric_inference_grid_1e9 = curve_metric_to_startend(inference_grid_1e9, dist_2 = bootstrap_counts_mean['A'].copy(), to_start = False, max_L = 100, normalize=True).unstack().transpose()
metric_inference_grid_1e9_final = metric_inference_grid_1e9.apply(lambda x: x[x.notnull()].values[-1])

In [ ]:
fig = make_subplots(rows = 2, cols = 3, shared_xaxes = True, shared_yaxes = True, subplot_titles = ['<i>m</i>=1','<i>m</i>=2','<i>m</i>=4', '<i>m</i>=8','<i>m</i>=16','<i>m</i>=32'], vertical_spacing = 0.12, horizontal_spacing = 0.05, row_heights = [0.5, 0.5], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for mult in [1,2,4]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = metric_inference_grid_1e9_final[mult].index.get_level_values(0), y = metric_inference_grid_1e9_final[mult].index.get_level_values(1), z=np.log10(metric_inference_grid_1e9_final[mult]), coloraxis='coloraxis1'), row = 1, col = col_count)
col_count = 0
for mult in [8,16,32]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = metric_inference_grid_1e9_final[mult].index.get_level_values(0), y = metric_inference_grid_1e9_final[mult].index.get_level_values(1), z=np.log10(metric_inference_grid_1e9_final[mult]), coloraxis='coloraxis1'), row = 2, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(title = 'log<sub>10</sub>(metric)', len=1), cmin = np.log10(metric_inference_grid_1e9.loc[metric_inference_grid_1e9.index.max()].min()), cmax= 3.55))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 580, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,3.5], dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,3.5], dtick = 2, col = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,3.5], dtick = 2, col = 2, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,3.5], dtick = 2, col = 3, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.add_shape(type="rect", x0=2.369-0.745, y0=2.879-0.53, x1=2.369+0.745, y1=2.879+0.53, line=dict(color='rgba(0,255,255,0.85)'), row = 2, col = 1)
fig.add_shape(type="rect", x0=2.369-0.745, y0=2.879-0.53, x1=2.369+0.745, y1=2.879+0.53, line=dict(color='rgba(0,255,255,0.85)'), row = 2, col = 2)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/figS8c.svg')
fig.write_image('plots/figS8c.pdf')
fig.write_image('plots/figS8c.png', scale = 10)

### Grid with stochastics

In [ ]:
folder_stoch = 'simulations/grid_A_L9_subonlystart_prospeedup_200_stoch/'
grid_files = os.listdir(folder_stoch)
grid_files_A = [file for file in grid_files if 'Adist' in file]

parameter_info_stoch = pd.Series(grid_files_A).str.split('_', expand = True)[[2,3,4,6,7,10,11,12]]
parameter_info_stoch.index = grid_files_A
parameter_info_stoch[2] = parameter_info_stoch[2].str.split('bins', expand = True)[1].astype(int)
parameter_info_stoch[3] = parameter_info_stoch[3].str.split('sp1e', expand = True)[1].astype(int)
#parameter_info_stoch[4] = parameter_info_stoch[4].str.split('rounds1e', expand = True)[1].astype(int)
parameter_info_stoch[6] = parameter_info_stoch[6].str.split('L', expand = True)[1].astype(int)
parameter_info_stoch[7] = parameter_info_stoch[7].str.split('x', expand = True)[1].astype(float)
parameter_info_stoch[10] = parameter_info_stoch[10].astype(float)
parameter_info_stoch[11] = parameter_info_stoch[11].astype(float)
parameter_info_stoch[12] = parameter_info_stoch[12].str.split('.pickle', expand = True)[0]
parameter_info_stoch.columns = ['bins', 'speedup (1e)', 'rounds (1e)', 'L_mult', 'mult', 'exp', 'con', 'start']

In [ ]:
parameters_inference_cols = parameter_info_stoch.groupby(['speedup (1e)', 'rounds (1e)', 'mult', 'exp', 'con', 'start'])['con'].count().index
inference_grid_stoch = dict()
for speedup, rounds, mult, exp, con, start in parameters_inference_cols:
    file = 'Adist_A_bins200_sp1e0_prospeedup_mult_L9_x'+str(mult)+'_extend_pl_'+str(exp)+'_'+str(con)+'_'+start+'_stochastics_1.pickle'
    inference_grid_stoch[(mult, exp, con)] = pd.read_pickle(folder_stoch + file)
    inference_grid_stoch[(mult, exp, con)].columns = inference_grid_stoch[(mult, exp, con)].columns * (10**speedup)
inference_grid_stoch = pd.concat(inference_grid_stoch, axis=1).sort_index(axis=1)

In [ ]:
inference_grid_stoch_final = inference_grid_stoch.T.groupby(level=[0,1,2]).tail(1).transpose()
inference_grid_stoch_final.columns = inference_grid_stoch_final.columns.droplevel(3)

In [ ]:
metric_inference_grid_stoch = curve_metric_to_startend(inference_grid_stoch, dist_2 = bootstrap_counts_mean['A'].copy(), to_start = False, max_L = 200, normalize=True).unstack().transpose()
metric_inference_grid_stoch_final = metric_inference_grid_stoch.apply(lambda x: x[x.notnull()].values[-1])

In [ ]:
# note: missing values are exceeding int32 limitation in the poisson calculations

In [ ]:
fig = make_subplots(rows = 2, cols = 3, shared_xaxes = True, shared_yaxes = True, subplot_titles = ['<i>m</i>=1','<i>m</i>=2','<i>m</i>=4', '<i>m</i>=8','<i>m</i>=16','<i>m</i>=32'], vertical_spacing = 0.12, horizontal_spacing = 0.05, row_heights = [0.5, 0.5], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for mult in [1,2,4]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = metric_inference_grid_stoch_final[mult].index.get_level_values(0), y = metric_inference_grid_stoch_final[mult].index.get_level_values(1), z=np.log10(metric_inference_grid_stoch_final[mult]), coloraxis='coloraxis1'), row = 1, col = col_count)
col_count = 0
for mult in [8,16,32]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = metric_inference_grid_stoch_final[mult].index.get_level_values(0), y = metric_inference_grid_stoch_final[mult].index.get_level_values(1), z=np.log10(metric_inference_grid_stoch_final[mult]), coloraxis='coloraxis1'), row = 2, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(title = 'log<sub>10</sub>(metric)', len=1), cmin = np.log10(metric_inference_grid_stoch.loc[metric_inference_grid_stoch.index.max()].min()), cmax= 3.55))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 580, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,2.5], dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,2.5], dtick = 2, col = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,2.5], dtick = 2, col = 2, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,2.5], dtick = 2, col = 3, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/figS8a.svg')
fig.write_image('plots/figS8a.pdf')
fig.write_image('plots/figS8a.png', scale = 10)

### Stochastics on top solution

In [ ]:
grid_files_stoch = os.listdir(folder + 'stoch/')
grid_files_A_stoch = [file for file in grid_files_stoch if file.startswith('Adist')]

parameter_info_stoch = pd.Series(grid_files_A_stoch).str.split('_', expand = True)[[2,7,10,11,14]]
parameter_info_stoch.index = grid_files_A_stoch
parameter_info_stoch[2] = parameter_info_stoch[2].str.split('bins', expand = True)[1].astype(int)
parameter_info_stoch[7] = parameter_info_stoch[7].str.split('x', expand = True)[1].astype(float)
parameter_info_stoch[10] = parameter_info_stoch[10]#.astype(float)
parameter_info_stoch[11] = parameter_info_stoch[11]#.astype(float)
parameter_info_stoch[14] = parameter_info_stoch[14].str.split('.pickle', expand = True)[0]

parameter_info_stoch.columns = ['bins', 'mult', 'exp', 'con', 'stochastics']

In [ ]:
motif = 'A'
parameter_info_stoch_cols = parameter_info_stoch.groupby(['bins', 'mult', 'exp', 'con', 'stochastics'])['bins'].count().index
best_stoch = dict()
for bins, mult, exp, con, stochastics in parameter_info_stoch_cols:
    file = 'Adist_A_bins'+str(bins)+'_sp1e0_prospeedup_mult_L9_x'+str(mult)+'_extend_pl_'+str(exp)+'_'+str(con)+'_subonlystart_stochastics_'+str(stochastics)+'.pickle'
    best_stoch[(motif, mult, float(exp), float(con), stochastics)] = pd.read_pickle(folder + 'stoch/' + file)
    best_stoch[(motif, mult, float(exp), float(con), stochastics)].columns = best_stoch[(motif, mult, float(exp), float(con), stochastics)].columns * (10**speedup)
best_stoch = pd.concat(best_stoch, axis=1).sort_index(axis=1)

In [ ]:
best_stoch_final = best_stoch.T.groupby(level=[0,1,2,3,4]).tail(1).transpose()

In [ ]:
def make_fig_stochastics(motif, mult, exp, con, timepoint = metric_inference_grid.index.max(), normalize = False):
    fig_stochastics = go.Figure()
    current = best_stoch_final[motif][mult][exp][con]
    maxL = (inference_grid_final[mult][exp][con] >= bootstrap_counts_mean[motif].min()).idxmin() +1
    leg = True
    if normalize == False:
        for run in current.columns:
            fig_stochastics.add_trace(go.Scatter(x = (current[run].index+1), y = current[run].replace(0, np.nan), mode = 'lines', line = dict(width = 1, color = plotly.colors.DEFAULT_PLOTLY_COLORS[0]), opacity = 0.1, legendgroup = 'med', name = 'computational model (stochastics)', showlegend = leg))
            leg = False
        fig_stochastics.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index * len(motif), y = bootstrap_counts_mean[motif], line = dict(color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))
        fig_stochastics.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index * len(motif), y = bootstrap_counts_max[motif], line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
        fig_stochastics.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index * len(motif), y = bootstrap_counts_min[motif], fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
        fig_stochastics.add_trace(go.Scatter(x = (inference_grid_final[mult][exp][con].index+1) * len(motif), y = inference_grid_final[mult][exp][con], mode = 'lines', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1]), legendgroup = 'med', name = 'computational model (deterministic)'))
        fig_stochastics.update_yaxes(type = 'log', title = 'counts', range = [-0.05,9.05], tickformat = '1.0e', dtick = 2)
    if normalize == True:
        for run in current.columns:
            fig_stochastics.add_trace(go.Scatter(x = (current[run].index+1)[:maxL], y = current[run][:maxL] / current[run].sum(), mode = 'lines', line = dict(width = 1, color = plotly.colors.DEFAULT_PLOTLY_COLORS[0]), opacity = 0.05, legendgroup = 'med', name = 'computational model (stochastics)', showlegend = leg))
            leg = False
        fig_stochastics.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index * len(motif), y = bootstrap_counts_mean[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))
        fig_stochastics.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index * len(motif), y = bootstrap_counts_max[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
        fig_stochastics.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index * len(motif), y = bootstrap_counts_min[motif] / bootstrap_counts_mean[motif].sum(), fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
        fig_stochastics.add_trace(go.Scatter(x = (inference_grid_final[mult][exp][con].index+1)[:maxL] * len(motif), y = inference_grid_final[mult][exp][con][:maxL] / inference_grid_final[mult][exp][con].sum(), mode = 'lines', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1]), legendgroup = 'med', name = 'computational model (deterministic)'))
        fig_stochastics.update_yaxes(type = 'log', title = 'frequency', range = [-9.05,0], tickformat = '1.0e', dtick = 2)
    fig_stochastics.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [0,2], tickvals = [1,2,5,10,20,50], gridcolor = 'rgba(0,0,0,0.2)')
    fig_stochastics.update_yaxes(gridcolor = 'rgba(0,0,0,0.2)')
    fig_stochastics.update_layout(width = 800, height = 330, font=dict(family = 'Arial', size = 20), margin={'t':20,'l':80,'b':50,'r':10})
    fig_stochastics.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=1.1))
    
    return fig_stochastics

In [ ]:
fig = make_fig_stochastics('A', 2,1.5,1.8, normalize=True)
fig.show()

In [ ]:
fig.write_image('plots/figS8b.svg')
fig.write_image('plots/figS8b.pdf')
fig.write_image('plots/figS8b.png', scale = 10)

### Genome size plot

In [ ]:
genome_size = inference_grid.mul(inference_grid.index+1, axis=0).sum() #+ inference_grid_B.mul(inference_grid_B.index+1, axis=0).sum()
genome_size_final = inference_grid_final.mul(inference_grid_final.index+1, axis=0).sum()

In [ ]:
fig = make_subplots(rows = 2, cols = 3, shared_xaxes = True, shared_yaxes = True, subplot_titles = ['<i>m</i>=1','<i>m</i>=2','<i>m</i>=4', '<i>m</i>=8','<i>m</i>=16','<i>m</i>=32'], vertical_spacing = 0.12, horizontal_spacing = 0.05, row_heights = [0.5, 0.5], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for mult in [1,2,4]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = genome_size_final.loc[mult].index.get_level_values(0), y = genome_size_final.loc[mult].index.get_level_values(1), z=np.log10(genome_size_final.replace(0, 1e50).loc[mult]), coloraxis='coloraxis1'), row = 1, col = col_count)
col_count = 0
for mult in [8,16,32]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = genome_size_final.loc[mult].index.get_level_values(0), y = genome_size_final.loc[mult].index.get_level_values(1), z=np.log10(genome_size_final.replace(0, 1e50).loc[mult]), coloraxis='coloraxis1'), row = 2, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(title = 'log<sub>10</sub>(L<sub>genome</sub>)', len=1), cmin = 9.3, cmax= 50))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 580, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,5], dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,5], dtick = 2, col = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,5], dtick = 2, col = 2, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_xaxes(range = [0,5], dtick = 2, col = 3, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/fig_S6.png', scale=10)
fig.write_image('plots/fig_S6.pdf')

In [ ]:
# log10 of starting genome size
np.log10((inference_grid[1][1][1][0] * (inference_grid.index+1)).sum())

## Poisson error calculations

In [ ]:
grid_files_poisson = os.listdir(folder + 'poisson/')
grid_files_A_poisson = [file for file in grid_files_poisson if file.startswith('Adist')]

parameter_info_poisson = pd.Series(grid_files_A_poisson).str.split('_', expand = True)[[2,3,4,7,10,11,12,13]]
parameter_info_poisson.index = grid_files_A_poisson
parameter_info_poisson[2] = parameter_info_poisson[2].str.split('bins', expand = True)[1].astype(int)
parameter_info_poisson[3] = parameter_info_poisson[3].str.split('sp1e', expand = True)[1].astype(int)
parameter_info_poisson[7] = parameter_info_poisson[7].str.split('x', expand = True)[1].astype(float)
parameter_info_poisson[10] = parameter_info_poisson[10]#.astype(float)
parameter_info_poisson[11] = parameter_info_poisson[11]#.astype(float)
parameter_info_poisson[12] = parameter_info_poisson[12].str.split('boot', expand = True)[1].astype(int)
parameter_info_poisson[13] = parameter_info_poisson[13].str.split('.pickle', expand = True)[0]

parameter_info_poisson.columns = ['bins', 'speedup (1e)', 'rounds (1e)', 'mult', 'exp', 'con', 'boot', 'start']

In [ ]:
len(grid_files_A_poisson)

In [ ]:
motif = 'A'
parameter_info_poisson_cols = parameter_info_poisson.groupby(['bins', 'speedup (1e)', 'rounds (1e)', 'mult', 'exp', 'con', 'boot', 'start'])['bins'].count().index
best_poisson = dict()
for bins, speedup, rounds, mult, exp, con, boot, start in parameter_info_poisson_cols:
    file = 'Adist_A_bins200_sp1e0_prospeedup_mult_L9_x'+str(mult)+'_extend_pl_'+str(exp)+'_'+str(con)+'_boot'+str(boot)+'_'+start+'.pickle'
    best_poisson[(motif, mult, float(exp), float(con), boot)] = pd.read_pickle(folder + 'poisson/' + file)
    best_poisson[(motif, mult, float(exp), float(con), boot)].columns = best_poisson[(motif, mult, float(exp), float(con), boot)].columns * (10**speedup)
best_poisson = pd.concat(best_poisson, axis=1).sort_index(axis=1)

In [ ]:
best_poisson_final = best_poisson.T.groupby(level=[0,1,2,3,4]).tail(1).transpose()

In [ ]:
def make_fig_boot(motif, mult, exp, con, timepoint = metric_inference_grid.index.max(), normalize = False):
    fig_boot = go.Figure()
    current = best_poisson_final[motif][mult][exp][con]
    maxL = (inference_grid_final[mult][exp][con] >= bootstrap_counts_mean[motif].min()).idxmin() +1
    if normalize == False:
        fig_boot.add_trace(go.Scatter(x = (inference_grid_final[mult][exp][con].index+1)[:maxL] * len(motif), y = inference_grid_final[mult][exp][con][:maxL], legendgroup = 'med', name = 'computational model'))
        fig_boot.add_trace(go.Scatter(x = (current.index+1)[:maxL] * len(motif), y = current.quantile(0.975, axis=1)[:maxL], line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'med', showlegend = False, name = 'bootstrap 95%'))
        fig_boot.add_trace(go.Scatter(x = (current.index+1)[:maxL] * len(motif), y = current.quantile(0.025, axis=1)[:maxL], fill='tonexty', fillcolor = 'rgba(31, 119, 180,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'med', showlegend = False, name = 'bootstrap 5%'))
        
        fig_boot.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index * len(motif), y = bootstrap_counts_mean[motif], line = dict(color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))
        fig_boot.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index * len(motif), y = bootstrap_counts_max[motif], line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
        fig_boot.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index * len(motif), y = bootstrap_counts_min[motif], fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
        fig_boot.update_yaxes(type = 'log', title = 'counts', range = [-0.05,9.05], tickformat = '1.0e', dtick = 2)
    if normalize == True:
        fig_boot.add_trace(go.Scatter(x = (inference_grid_final[mult][exp][con].index+1)[:maxL] * len(motif), y = inference_grid_final[mult][exp][con][:maxL] / inference_grid_final[mult][exp][con].sum(), legendgroup = 'med', name = 'computational model'))
        fig_boot.add_trace(go.Scatter(x = (current.index+1)[:maxL] * len(motif), y = current.quantile(0.975, axis=1)[:maxL] / current.median(axis=1).sum(), line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'med', showlegend = False, name = 'bootstrap 95%'))
        fig_boot.add_trace(go.Scatter(x = (current.index+1)[:maxL] * len(motif), y = current.quantile(0.025, axis=1)[:maxL] / current.median(axis=1).sum(), fill='tonexty', fillcolor = 'rgba(31, 119, 180,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'med', showlegend = False, name = 'bootstrap 5%'))
        
        fig_boot.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index * len(motif), y = bootstrap_counts_mean[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))
        fig_boot.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index * len(motif), y = bootstrap_counts_max[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
        fig_boot.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index * len(motif), y = bootstrap_counts_min[motif] / bootstrap_counts_mean[motif].sum(), fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
        fig_boot.update_yaxes(type = 'log', title = 'frequency', range = [-9.05,0], tickformat = '1.0e', dtick = 2)
    fig_boot.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [0,2], tickvals = [1,2,5,10,20,50], gridcolor = 'rgba(0,0,0,0.2)')
    fig_boot.update_yaxes(gridcolor = 'rgba(0,0,0,0.2)')

    fig_boot.update_layout(font=dict(family = 'Arial', size = 13), margin={'t':20,'l':60,'b':40,'r':10})

    fig_boot.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
    
    return fig_boot

In [ ]:
fig_boot = make_fig_boot('A', 2, 1.5, 1.8, normalize = True)
fig_boot.update_layout(height = 250, width = 470, margin={'t':20,'l':55,'b':35,'r':10})       

In [ ]:
fig_boot.write_image('plots/fig2c.svg')
fig_boot.write_image('plots/fig2c.pdf')
fig_boot.write_image('plots/fig2c.png', scale = 10)

In [ ]:
def poisson_metric(motif, mult, exp, con, timepoint = metric_inference_grid.index.max(), normalize = False, max_L = 200):
    poisson_metric = dict()
    poisson_current = best_poisson_final[motif][mult][exp][con]
    for boot in poisson_current.columns.levels[0]:
        poisson_metric[boot] = curve_metric_to_startend(poisson_current[boot], dist_2 = bootstrap_counts[boot][motif].copy(), to_start=False, max_L = 200, normalize = normalize)
    poisson_metric = pd.concat(poisson_metric).sort_values().head(len(poisson_metric) - int(0.05 * len(poisson_metric)))   # one-sided 95%
    return poisson_metric.median(), poisson_metric.max(), poisson_metric.min()

## Plot diagonals

In [ ]:
def fig_add_paramlist(fig, param_list, group_name, colorlist = plotly.colors.cyclical.Phase, motif = 'A', colornum = 0, df = inference_grid_final, opacity = 0.5, normalize = False, show_CHM13 = False):
    group = df.reindex(param_list.set_index(['mult', 'exp', 'con']).index, axis=1)
    group.columns = list(range(len(group.columns)))
    group = group[group >1].fillna(1,limit=1, axis=0)
    group.loc[len(group)+1] = np.nan
    group_index = np.array(([group.index+1]*200)).flatten()
    if normalize == True:
        group = group.div(group.sum())
    group = pd.Series(group.transpose().to_numpy().flatten())
    
    fig.add_trace(go.Scatter(x = group_index, y = group, mode = 'lines', line = dict(width = 1.5, color = colorlist[colornum] if colornum != -1 else 'rgba(120,120,120,0.2)'), name = group_name, connectgaps = False, opacity = opacity))

    if show_CHM13 == True:
        if normalize == True:
            fig.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index, y = bootstrap_counts_mean[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,1)', width = 3), legendgroup = 'ci', name = 'T2T-CHM13'))
            fig.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index, y = bootstrap_counts_max[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
            fig.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index, y = bootstrap_counts_min[motif] / bootstrap_counts_mean[motif].sum(), fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
        else:
            fig.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index, y = bootstrap_counts_mean[motif], line = dict(color = 'rgba(0,0,0,1)', width = 3), legendgroup = 'ci', name = 'T2T-CHM13'))
            fig.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index, y = bootstrap_counts_max[motif], line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
            fig.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index, y = bootstrap_counts_min[motif], fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
    fig.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [0,2], gridcolor = 'rgba(0,0,0,0.2)')
    if normalize == True:
        fig.update_yaxes(type = 'log', title = 'frequency', range = [-9,0], gridcolor = 'rgba(0,0,0,0.2)', tickformat = '1.0e', dtick = 2)
    else:
        fig.update_yaxes(type = 'log', title = 'counts', range = [0,11], gridcolor = 'rgba(0,0,0,0.2)', tickformat = '1.0e', dtick = 2)
    fig.update_layout(font=dict(family = 'Arial', size = 16), height = 300, width = 780, margin={'t':20,'l':60,'b':40,'r':10}) 

In [ ]:
# ignore lower left corner
mult = 8
corner_size = 1
diagonal_groups = dict()
for diag in [0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1, -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2, -2.1, -2.2, -2.3, -2.4, -2.5, -2.6, -2.7, -2.8, -2.9, -3, -3.1, -3.2, -3.3, -3.4, -3.5, -3.6, -3.7, -3.8, -3.9, -4, -4.1, -4.2, -4.3, -4.4, -4.5, -4.6, -4.7, -4.8, -4.9]:#, -5]:
    diagonal_groups[diag] = parameter_info.loc[(parameter_info['mult'] == mult) & (parameter_info['exp'] - parameter_info['con'] >= diag-0.01) & (parameter_info['exp'] - parameter_info['con'] <= diag+0.01)  & ((parameter_info['exp'] >corner_size) | (parameter_info['con'] >corner_size))].copy()
    if len(diagonal_groups[diag]) == 0:
        del diagonal_groups[diag]

diagonal_groups_exp = dict()
for diag in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]:#, 5]:
    diagonal_groups_exp[diag] = parameter_info.loc[(parameter_info['mult'] == mult) & (parameter_info['exp'] - parameter_info['con'] >= diag-0.01) & (parameter_info['exp'] - parameter_info['con'] <= diag+0.01)  & ((parameter_info['exp'] >corner_size) | (parameter_info['con'] >corner_size))].copy()
    if len(diagonal_groups_exp[diag]) == 0:
        del diagonal_groups_exp[diag]

In [ ]:
current_colorscale = plotly.colors.sample_colorscale(plotly.colors.sequential.Rainbow, np.linspace(0,len(diagonal_groups.keys())) / len(diagonal_groups.keys()))

In [ ]:
# illustrative figure
fig_phases = go.Figure()
for diag in list(diagonal_groups_exp.keys())[:-1]:
    fig_phases.add_trace(go.Scatter(x = diagonal_groups_exp[diag]['exp'], y = diagonal_groups_exp[diag]['con'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 6, color = 'rgba(120,120,120,0.65)')))
colornum = 0
for diag in diagonal_groups:
    fig_phases.add_trace(go.Scatter(x = diagonal_groups[diag]['exp'], y = diagonal_groups[diag]['con'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 6, color = current_colorscale[colornum])))
    colornum +=1

fig_phases.add_trace(go.Scatter(x = [0,5], y = [1.5, 6.5], line = dict(color = 'white', width = 3.5, dash = 'dash'), opacity = 0.8, mode = 'lines', showlegend = False))
fig_phases.add_trace(go.Scatter(x = [0,4.5], y = [0.5, 5], line = dict(color = 'white', width = 3.5, dash = 'dash'), opacity = 0.8, mode = 'lines', showlegend = False))
fig_phases.add_trace(go.Scatter(x = [0,5], y = [0, 5], line = dict(color = 'white', width = 3.5, dash = 'dash'), opacity = 0.8, mode = 'lines', showlegend = False))

fig_phases.add_trace(go.Scatter(x = [0], y = [3], marker = dict(color = 'red', size = 8, line_width = 1, line_color = 'white', symbol = 'circle'), mode = 'markers', name = 'i'))
fig_phases.add_trace(go.Scatter(x = [1], y = [2], marker = dict(color = 'red', size = 9, line_width = 1, line_color = 'white', symbol = 'diamond'), mode = 'markers', name = 'ii'))
fig_phases.add_trace(go.Scatter(x = [1.3], y = [1.7], marker = dict(color = 'red', size = 8, line_width = 1, line_color = 'white', symbol = 'square'), mode = 'markers', name = 'iii'))
fig_phases.add_trace(go.Scatter(x = [1.5], y = [1.5], marker = dict(color = 'red', size = 11, line_width = 1, line_color = 'white', symbol = 'triangle-up'), mode = 'markers', name = 'iv'))
fig_phases.add_trace(go.Scatter(x = [2.5], y = [0.5], marker = dict(color = 'red', size = 11, line_width = 1, line_color = 'white', symbol = 'triangle-down'), mode = 'markers', name = 'v'))

fig_phases.update_layout(font=dict(family = 'Arial', size = 14), height = 250, width = 270, margin={'t':30,'l':35,'b':40,'r':0})
fig_phases.update_xaxes(range = [-0.2, 3.05], dtick = 1, title = 'expansion power 𝜏<sub>𝜀</sub>', gridcolor = 'rgba(0,0,0,0.2)')
fig_phases.update_yaxes(range = [0, 3.1], dtick = 1, title = 'contraction power 𝜏<sub>𝜅</sub>', gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="x", scaleratio=1)
fig_phases.update_layout(legend=dict(indentation = -15))
fig_phases.show()

In [ ]:
fig_phases.write_image('plots/fig3a.svg')
fig_phases.write_image('plots/fig3a.pdf')

In [ ]:
# ignore lower left corner
for mult, corner_size in zip([1,2,4,8,16,32], [1.5, 1.4, 1.3, 1.2, 1.1, 1]):
    diagonal_groups = dict()
    for diag in [0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1, -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2, -2.1, -2.2, -2.3, -2.4, -2.5, -2.6, -2.7, -2.8, -2.9, -3, -3.1, -3.2, -3.3, -3.4, -3.5, -3.6, -3.7, -3.8, -3.9, -4, -4.1, -4.2, -4.3, -4.4, -4.5, -4.6, -4.7, -4.8, -4.9]:
        diagonal_groups[diag] = parameter_info.loc[(parameter_info['mult'] == mult) & (parameter_info['exp'] - parameter_info['con'] >= diag-0.01) & (parameter_info['exp'] - parameter_info['con'] <= diag+0.01)  & ((parameter_info['exp'] >corner_size) | (parameter_info['con'] >corner_size))].copy()
        diagonal_groups[diag]['round'] = 1e9
        if len(diagonal_groups[diag]) == 0:
            del diagonal_groups[diag]
    
    diagonal_groups_exp = dict()
    for diag in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]:
        diagonal_groups_exp[diag] = parameter_info.loc[(parameter_info['mult'] == mult) & (parameter_info['exp'] - parameter_info['con'] >= diag-0.01) & (parameter_info['exp'] - parameter_info['con'] <= diag+0.01)  & ((parameter_info['exp'] >corner_size) | (parameter_info['con'] >corner_size))].copy()
        diagonal_groups_exp[diag]['round'] = 1e9
        if len(diagonal_groups_exp[diag]) == 0:
            del diagonal_groups_exp[diag]

    fig_phases = go.Figure()
    for diag in list(diagonal_groups_exp.keys())[:-1]:
        fig_phases.add_trace(go.Scatter(x = diagonal_groups_exp[diag]['exp'], y = diagonal_groups_exp[diag]['con'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 4.5, color = 'rgba(120,120,120,0.65)')))
    colornum = 0
    for diag in diagonal_groups:
        fig_phases.add_trace(go.Scatter(x = diagonal_groups[diag]['exp'], y = diagonal_groups[diag]['con'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 4.5, color = current_colorscale[colornum])))
        colornum +=1
    fig_phases.update_layout(title = '<i>m</i>='+str(mult), font=dict(family = 'Arial', size = 14), height = 220, width = 220, margin={'t':30,'l':35,'b':40,'r':10})
    fig_phases.update_xaxes(range = [0,5.1], dtick = 1, title = 'expansion power 𝜏<sub>𝜀</sub>', gridcolor = 'rgba(0,0,0,0.2)')
    fig_phases.update_yaxes(range = [0,5.1], dtick = 1, title = 'contraction power 𝜏<sub>𝜅</sub>', gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="x", scaleratio=1)
    fig_phases.show()
    
    fig = go.Figure()
    for diag in list(diagonal_groups_exp.keys())[:-1]:
        fig_add_paramlist(fig, diagonal_groups_exp[diag], group_name = str(diag), colornum = -1, normalize = True, show_CHM13 = False)
    colornum = 0
    for diag in list(diagonal_groups.keys())[:-1]:
        fig_add_paramlist(fig, diagonal_groups[diag], group_name = str(diag), colorlist = current_colorscale, colornum = colornum, opacity = 0.4, normalize = True, show_CHM13 = False)
        colornum +=1
    for diag in list(diagonal_groups.keys())[-1:]:
        fig_add_paramlist(fig, diagonal_groups[diag], group_name = str(diag), colorlist = current_colorscale, colornum = colornum, opacity = 0.4, normalize = True, show_CHM13 = True)
    fig.update_xaxes(range=[0,2.01], tickvals = [1,2,5,10,20,50,100])
    fig.update_layout(font=dict(family = 'Arial', size = 14), height = 250, width = 520, margin={'t':30,'l':60,'b':40,'r':10}, showlegend = False)
    fig.show()
    
    fig_phases.write_image('plots/figS5a_mult'+str(mult)+'.pdf')
    fig.write_image('plots/figS5b_mult'+str(mult)+'.pdf')

## flux plots for analytics comparison

In [ ]:
# plots for Fig. 3c

In [ ]:
def plot_flux_combinations(comb_list, mult = 4, separate = False, show_plot = True, showlegend = True, write = False, cut_at_n1 = False, norm = True, legend_size = False, A_bins = 200):
    for parameters in comb_list:
        input_df_B = pd.read_pickle(folder + 'Bdist_A_bins200_sp1e0_prospeedup_mult_L9_x'+str(float(mult))+'_extend_pl_'+str(float(parameters[0]))+'_'+str(float(parameters[1]))+'_subonlystart.pickle')
        fig_flux, last_int_bin = plot_relative_processes(mult,parameters[0],parameters[1], [inference_grid_final[mult][parameters[0]][parameters[1]], input_df_B[input_df_B.columns.max()]], norm=norm, separate=separate, cut_at_n1=cut_at_n1, A_bins=A_bins)
        if cut_at_n1 == False:
            fig_flux.update_xaxes(range = [0.5, 99])
        fig_flux.update_xaxes(dtick = 10)
        fig_flux.update_layout(font=dict(family = 'Arial', size = 12), title = str((mult,parameters[0],parameters[1])), height = 210, width = 550, margin={'t':60,'l':60,'b':40,'r':10}, showlegend = showlegend)
        if legend_size == True:
            fig_flux.update_layout(font=dict(family = 'Arial', size = 12), title = str((mult,parameters[0],parameters[1])), height = 500, width = 500, margin={'t':60,'l':60,'b':40,'r':10}, showlegend = showlegend)
        if cut_at_n1 == True:
            if last_int_bin > 10:
                fig_flux.update_layout(width = 80 + 7*last_int_bin)
            else:
                fig_flux.update_xaxes(dtick = 50)
        if show_plot == True:
            fig_flux.show()
        if write != False:
            if separate == False:
                if cut_at_n1 == 'sup':
                    fig_flux.write_image('plots/fig_sup_flux_'+str(mult)+'_'+str(parameters[0])+'_'+str(parameters[1])+'.'+ write)
                else:
                    fig_flux.write_image('plots/fig_flux_'+str(mult)+'_'+str(parameters[0])+'_'+str(parameters[1])+'.'+ write)
            else:
                if cut_at_n1 == 'sup':
                    fig_flux.write_image('plots/fig_sup_flux_separated'+str(mult)+'_'+str(parameters[0])+'_'+str(parameters[1])+'.'+ write)
                else:
                    fig_flux.write_image('plots/fig_flux_separated'+str(mult)+'_'+str(parameters[0])+'_'+str(parameters[1])+'.'+ write)

In [ ]:
parameter_combination_examples = [[0,3], [1,2], [1.3, 1.7], [1.5, 1.5], [2.5, 0.5]]
plot_flux_combinations(parameter_combination_examples, mult = 8, showlegend = False, separate = False, cut_at_n1=True, A_bins=200, write = 'pdf')

#### additional plots for Supplemental Note 1

In [ ]:
parameter_combination_examples = [[0,3], [0.5,2.5], [0.8,2.2], [1,2], [1.1,1.9], [1.2,1.8], [1.3, 1.7], [1.4,1.6], [1.5, 1.5], [2.5, 0.5]]
plot_flux_combinations(parameter_combination_examples, mult = 8, showlegend = False, separate = False, cut_at_n1='sup', A_bins=200, write = 'pdf')

In [ ]:
parameter_combination_examples = [[0,3], [0.5,2.5], [0.8,2.2], [1,2], [1.1,1.9], [1.2,1.8], [1.3, 1.7], [1.4,1.6], [1.5, 1.5], [2.5, 0.5]]
plot_flux_combinations(parameter_combination_examples, mult = 8, showlegend = False, separate = True, cut_at_n1='sup', A_bins=200, write = 'pdf')

In [ ]:
def plot_net_flux_lineplot(comb_list, mult = 8, show_plot = True, show_sum = False, show_legend = True, write = False):
    for parameters in comb_list:
        input_df_B = pd.read_pickle(folder + 'Bdist_A_bins200_sp1e0_prospeedup_mult_L9_x'+str(float(mult))+'_extend_pl_'+str(float(parameters[0]))+'_'+str(float(parameters[1]))+'_subonlystart.pickle')

        data = plot_relative_processes(mult,parameters[0],parameters[1], [inference_grid_final[mult][parameters[0]][parameters[1]], input_df_B[input_df_B.columns.max()]], A_bins=200, norm = False, returndata = 'sum')
        data_ins = plot_relative_processes(mult,parameters[0],parameters[1], [inference_grid_final[mult][parameters[0]][parameters[1]], input_df_B[input_df_B.columns.max()]], A_bins=200, norm = False, returndata = 'in')
        data_outs = plot_relative_processes(mult,parameters[0],parameters[1], [inference_grid_final[mult][parameters[0]][parameters[1]], input_df_B[input_df_B.columns.max()]], A_bins=200, norm = False, returndata = 'out')
        Lmax = (inference_grid_final[mult][parameters[0]][parameters[1]] >= 1).idxmin()
        if Lmax < 10:
            Lmax = 200
        
        components_fission_in = data_ins.loc['substitution fission'] + data_ins.loc['non-motif insertion fission']
        components_fission_out = data_outs.loc['substitution fission'] + data_outs.loc['non-motif insertion fission']
        
        local_terms = data_ins.loc['expansion +1'] + data_ins.loc['contraction -1'] + (data_outs.loc['expansion +1'] + data_outs.loc['contraction -1'])
        local_terms_wsub = data_ins.loc['expansion +1'] + data_ins.loc['contraction -1'] + data_ins.loc['substitution +1'] + data_ins.loc['substitution -1'] + (data_outs.loc['expansion +1'] + data_outs.loc['contraction -1'] + data_outs.loc['substitution +1'] + data_outs.loc['substitution -1'])
        local_subonly = data_ins.loc['substitution +1'] + data_ins.loc['substitution -1'] + (data_outs.loc['substitution +1'] + data_outs.loc['substitution -1'])
        all_subonly = data_ins.loc['substitution +1'] + data_ins.loc['substitution -1'] + data_ins.loc['substitution fission'] + data_ins.loc['substitution fusion'] + (data_outs.loc['substitution +1'] + data_outs.loc['substitution -1'] + data_outs.loc['substitution fission'] + data_outs.loc['substitution fusion'])
        local_plus_fission_out = local_terms + components_fission_out
        local_plus_fission_all = local_terms + components_fission_in + components_fission_out
        all_terms = (data_ins + data_outs).sum()
        
        fig_components = go.Figure()
        fig_components.add_trace(go.Scatter(x = all_terms.index[:Lmax], y = all_terms[:Lmax], mode = 'lines', line = dict(color = 'rgba(1,1,1,0.8)', width = 3), name = 'computational model: all terms'))
        fig_components.add_trace(go.Scatter(x = local_terms.index[:Lmax], y = local_terms[:Lmax], mode = 'lines', line = dict(color = 'rgba(255,178,101,0.8)', width = 3), name = 'local terms (no fission, no fusion)'))
        fig_components.add_trace(go.Scatter(x = local_plus_fission_out.index[:Lmax], y = local_plus_fission_out[:Lmax], mode = 'lines', line = dict(color = 'rgba(255,101,101,0.8)', width = 3), name = 'no fission in, no fusion'))
        fig_components.add_trace(go.Scatter(x = local_plus_fission_all.index[:Lmax], y = local_plus_fission_all[:Lmax], mode = 'lines', line = dict(color = 'rgba(178,101,178,0.8)', width = 3), name = 'no fusion'))
        fig_components.update_xaxes(title = 'repeat length (nt)')
        yrange = max(0.05, all_terms[5:15].abs().max())
        fig_components.update_yaxes(title = 'net flux (mut. per gen.)', range = [-yrange, yrange])
        fig_components.update_layout(font=dict(family = 'Arial', size = 12), title = str((mult,parameters[0],parameters[1])), height = 210, width = 550, margin={'t':60,'l':60,'b':40,'r':10}, showlegend = show_legend)
        if show_plot == True:
            fig_components.show()
        if show_sum == True:
            print(all_terms.sum())
        if write != False:
            fig_components.write_image('plots/fig_netfluxlines_finaltimepoint_'+str(mult)+'_'+str(parameters[0])+'_'+str(parameters[1])+'.'+ write)

In [ ]:
plot_net_flux_lineplot(parameter_combination_examples, show_legend=False, write = 'pdf')
plot_net_flux_lineplot([[0,0]], show_legend=True, write = 'pdf')

## Plotting timescales

In [ ]:
# timepoint where metric value drops below the Poisson error of the best parameter combination, if the final timepoint is also below
timepoint_below = ((metric_inference_grid_1e9 < poisson_metric('A', 2, 1.5, 1.8, normalize=True)[1]).idxmax().mul((metric_inference_grid_1e9.loc[1e9] < poisson_metric('A', 2, 1.5, 1.8, normalize=True)[1]).astype(int)))

In [ ]:
all_consistent = metric_inference_grid_final.loc[metric_inference_grid_final < poisson_metric('A', 2, 1.5, 1.8, normalize=True)[1]].copy()

In [ ]:
fig = make_subplots(rows = 2, cols = 3, shared_xaxes = True, shared_yaxes = True, subplot_titles = ['<i>m</i>=1','<i>m</i>=2','<i>m</i>=4', '<i>m</i>=8','<i>m</i>=16','<i>m</i>=32'], vertical_spacing = 0.12, horizontal_spacing = 0.05, row_heights = [0.52, 0.48], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for mult in [1,2,4]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = all_consistent[mult].dropna().index.get_level_values(0), y = all_consistent[mult].dropna().index.get_level_values(1), z=[1]*len(all_consistent[mult].dropna().index), coloraxis='coloraxis2', hoverongaps=False, showlegend = False), row = 1, col = col_count)
    fig.add_trace(go.Heatmap(x = timepoint_below[mult].dropna().index.get_level_values(0), y = timepoint_below[mult].dropna().index.get_level_values(1), z=timepoint_below[mult].replace(0, np.nan), coloraxis='coloraxis1', hoverongaps=False), row = 1, col = col_count)
col_count = 0
for mult in [8,16,32]:
    col_count +=1
    fig.add_trace(go.Heatmap(x = all_consistent[mult].dropna().index.get_level_values(0), y = all_consistent[mult].dropna().index.get_level_values(1), z=[1]*len(all_consistent[mult].dropna().index), coloraxis='coloraxis2', hoverongaps=False, showlegend = False), row = 2, col = col_count)
    fig.add_trace(go.Heatmap(x = timepoint_below[mult].dropna().index.get_level_values(0), y = timepoint_below[mult].dropna().index.get_level_values(1), z=timepoint_below[mult].replace(0, np.nan), coloraxis='coloraxis1', hoverongaps=False), row = 2, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='sunset_r', cmin = 0, cmax = 1e8, colorbar = dict(title = 'generation', len=1)))
fig.update_layout(coloraxis2=dict(colorscale='greys', cmin = 0, cmax = 5, colorbar = dict(title = 'generation', len=1), showscale = False))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 580, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_xaxes(range = [0,5], dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_yaxes(range = [0,5], dtick = 2, row = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="x", scaleratio=1)
fig.update_yaxes(range = [0,5], dtick = 2, row = 2, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="x", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)

fig.add_shape(type="rect", x0=2.369-0.745, y0=2.879-0.53, x1=2.369+0.745, y1=2.879+0.53, line=dict(color='rgba(0,255,255,0.85)'), row = 2, col = 1)
fig.add_shape(type="rect", x0=2.369-0.745, y0=2.879-0.53, x1=2.369+0.745, y1=2.879+0.53, line=dict(color='rgba(0,255,255,0.85)'), row = 2, col = 2)
fig.show()

In [ ]:
fig.write_image('plots/fig2d.svg')
fig.write_image('plots/fig2d.pdf')
fig.write_image('plots/fig2d.png', scale=10)

## Substitution-only simulations

In [ ]:
grid_files_mutonly = os.listdir('simulations/subonly_output/')
grid_files_A_mutonly = [file for file in grid_files_mutonly if file.startswith('Adist')]

In [ ]:
parameter_info_mutonly = pd.Series(grid_files_A_mutonly).str.split('_', expand = True)[[1,2,3,4,6]]
parameter_info_mutonly.index = grid_files_A_mutonly
parameter_info_mutonly[2] = parameter_info_mutonly[2].str.split('bins', expand = True)[1].astype(int)
parameter_info_mutonly[3] = parameter_info_mutonly[3].str.split('sp1e', expand = True)[1].astype(int)
parameter_info_mutonly[4] = parameter_info_mutonly[4].str.split('rounds1e', expand = True)[1].astype(int)
parameter_info_mutonly[6] = parameter_info_mutonly[6].str.split('.pickle', expand = True)[0]
parameter_info_mutonly.columns = ['motif', 'bins', 'speedup (1e)', 'rounds (1e)', 'start']

In [ ]:
parameter_info_mutonly

In [ ]:
parameter_info_mutonly_cols = parameter_info_mutonly.groupby(['motif', 'bins', 'speedup (1e)', 'rounds (1e)', 'start'])['bins'].count().index
substitutions_only = dict()
for motif, bins, speedup, rounds, start in parameter_info_mutonly_cols:
    file = 'Adist_'+motif+'_bins'+str(bins)+'_sp1e'+str(speedup)+'_rounds1e'+str(rounds)+'_mutonly_'+start+'.pickle'
    substitutions_only[motif] = pd.read_pickle('simulations/subonly_output/' + file)
    substitutions_only[motif].columns = substitutions_only[motif].columns * (10**speedup)
substitutions_only = pd.concat(substitutions_only, axis=1).sort_index(axis=1)

In [ ]:
starting_subonlydist = dict()
for motif in ['A', 'AC', 'AAC', 'AAAC']:
    starting_subonlydist[motif] = substitutions_only[motif][1e10][:100]
starting_subonlydist = pd.concat(starting_subonlydist, axis=1)
starting_subonlydist.index +=1
starting_subonlydist.to_pickle('repeat_distributions/subonly_counts.pickle')

In [ ]:
subonly_counts = pd.read_pickle('repeat_distributions/subonly_counts.pickle')

In [ ]:
fig_mutonly = go.Figure()
for motif in ['A', 'AC', 'AAC', 'AAAC']:
    fig_mutonly.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index * len(motif), y = bootstrap_counts_mean[motif].loc[bootstrap_counts_mean[motif] >=1] / bootstrap_counts_mean[motif].loc[bootstrap_counts_mean[motif] >=1].sum(), opacity = 0.6, mode = 'lines', line = dict(width = 3), legendgroup = motif, name = motif))
for motif in ['A', 'AC', 'AAC', 'AAAC']:
    fig_mutonly.add_trace(go.Scatter(x = random_counts['A'][motif].index * len(motif), y = random_counts['A'][motif].loc[random_counts['A'][motif] >=1] / random_counts['A'][motif].loc[random_counts['A'][motif] >=1].sum(), opacity = 0.75, mode = 'lines', line = dict(dash = 'dot', width = 3), legendgroup = motif, name = 'shuffled'))
for motif in ['A', 'AC', 'AAC', 'AAAC']:
    sub_only = substitutions_only[motif][1e9]
    sub_only.index +=1
    sub_only = sub_only.loc[sub_only >=1]
    fig_mutonly.add_trace(go.Scatter(x = (sub_only.index) * len(motif), y = sub_only / sub_only.sum(), opacity = 0.5, mode = 'lines', line = dict(dash = 'dash', width = 3), legendgroup = motif, name = 'substitutions only'))
fig_mutonly.update_yaxes(type = 'log', title = 'frequency', range = [-9.05,0], tickformat = '1.0e', dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig_mutonly.update_xaxes(type = 'log', title = 'repeat length (nt)', range = [0,2.2], tickvals = [1,2,5,10,20,50,100], gridcolor = 'rgba(0,0,0,0.2)')
fig_mutonly.update_layout(font=dict(family = 'Arial', size = 14), height = 360, width = 700, margin={'t':20,'l':55,'b':35,'r':20}, colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:4])

fig_mutonly.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=1.2))

In [ ]:
fig_mutonly.write_image('plots/figS7.pdf')